# Analysis for DFProfiler

This is a simple analysis notebook for dfprofiler.

## Imports

In [1]:
import logging
import json
import dask
import os
from pathlib import Path
from glob import glob
import zindex_py as zindex

In [2]:

import dask.dataframe as dd
from dask.distributed import Client, LocalCluster, progress, wait, get_client
from dask.distributed import Future, get_client

## Project Variables

In [3]:
app_root = str(Path(os.getcwd()).parent)

In [4]:
logging.basicConfig(
    level=logging.DEBUG,
    handlers=[
        logging.StreamHandler(),
    ],
    format="%(asctime)s [%(levelname)s]: %(message)s in %(pathname)s:%(lineno)d",
)

## Setup Dask Local Cluster

In [5]:
workers=4
cluster = LocalCluster(n_workers=workers)  # Launches a scheduler and workers locally
client = Client(cluster)  # Connect to distributed cluster and override default
logging.info(f"Initialized Client with {workers} workers and link {client.dashboard_link}")

2024-07-21 05:54:28,067 [DEBUG]: Using selector: EpollSelector in /usr/lib/python3.10/asyncio/selector_events.py:54
2024-07-21 05:54:28,957 [INFO]: Initialized Client with 4 workers and link http://127.0.0.1:8787/status in /tmp/ipykernel_382455/1637310826.py:4


## Start Analysis

In [6]:

file=f"{app_root}/tests/output/simple_test_1MB_128K.pfw.gz"
file_pattern = glob(file)
file_pattern

['/home/cc/dfprofiler/tests/output/simple_test_1MB_128K.pfw.gz']

## Function to load trace data

In [7]:
def create_index(filename):
    index_file = f"{filename}.zindex"
    if not os.path.exists(index_file):
        status = zindex.create_index(filename, index_file=f"file:{index_file}",
                                     regex="id:\b([0-9]+)", numeric=True, unique=True, debug=False, verbose=False)
        logging.debug(f"Creating Index for {filename} returned {status}")
    return filename

def get_linenumber(filename):
    index_file = f"{filename}.zindex"
    line_number = zindex.get_max_line(filename, index_file=index_file, debug=False, verbose=False)
    logging.debug(f" The {filename} has {line_number} lines")
    return (filename, line_number)

def get_size(filename):
    conf = get_dft_configuration()
    if filename.endswith('.pfw'):
        size = os.stat(filename).st_size
    elif filename.endswith('.pfw.gz'):
        index_file = f"{filename}.zindex"
        line_number = zindex.get_max_line(filename, index_file=index_file,debug=False, verbose=False)
        size = line_number * 256
    logging.debug(f" The {filename} has {size/1024**3} GB size")
    return int(size)


def generate_line_batches(filename, max_line):
    batch_size = 16*1024
    for start in range(0, max_line, batch_size):
        end =  min((start + batch_size - 1) , (max_line - 1))
        logging.debug(f"Created a batch for {filename} from [{start}, {end}] lines")
        yield filename, start, end

def load_indexed_gzip_files(filename, start, end):
    index_file = f"{filename}.zindex"
    json_lines = zindex.zquery(filename, index_file=index_file,
                          raw=f"select a.line from LineOffsets a where a.line >= {start} AND a.line <= {end};", debug=False, verbose=False)
    logging.debug(f"Read {len(json_lines)} json lines for [{start}, {end}]")
    return json_lines

In [23]:
def load_profile(line):
    d = {}
    if line is not None and line !="" and len(line) > 0 and "[" != line[0] and line != "\n" :
        try:
            unicode_line = ''.join([i if ord(i) < 128 else '#' for i in line])
            val = json.loads(unicode_line)
            d["pid"] = val["pid"]
            d["tid"] = val["tid"]
            d["ts_us"] = int(val["ts"])
            if "args" in val:
                d["dur_sec"] = float(val["args"]["time"])
                d["freq"] = val["args"]["freq"]
                d["size_bytes"] = val["args"]["size_sum"]
            d["func_id"] = val["name"]
            d["cat"] = val["cat"]
        except Exception as error:
            logging.error(f"Processing {line} failed with {error}")
    return d

## Create Dask Dataframe

In [30]:
if len(file_pattern) > 0:
    dask.bag.from_sequence(file_pattern).map(create_index).compute()
    logging.info(f"Created index for {len(file_pattern)} files")
    total_size = dask.bag.from_sequence(file_pattern).map(get_size).sum()
    logging.info(f"Total size of all files are {total_size} bytes")
    max_line_numbers = dask.bag.from_sequence(file_pattern).map(get_linenumber).compute()
    logging.info(f"Max lines per file are {max_line_numbers}")
    json_line_delayed = []
    total_lines = 0
    for filename, max_line in max_line_numbers:
        total_lines += max_line
        for _, start, end in generate_line_batches(filename, max_line):
            json_line_delayed.append((filename, start, end))

    logging.info(f"Loading {len(json_line_delayed)} batches out of {len(file_pattern)} files and has {total_lines} lines overall")
    json_line_bags = []
    for filename, start, end in json_line_delayed:
        num_lines = end - start + 1
        json_line_bags.append(dask.delayed(load_indexed_gzip_files, nout=num_lines)(filename, start, end))
    json_lines = dask.bag.concat(json_line_bags)
    pfw_bag = json_lines.map(load_profile).filter(lambda x: "ts_us" in x)
    pfw_bag.take(1)

2024-07-21 06:01:00,652 [INFO]: Created index for 1 files in /tmp/ipykernel_382455/405840175.py:3
2024-07-21 06:01:00,658 [INFO]: Total size of all files are <dask.bag.core.Item object at 0x7d8a346ef130> bytes in /tmp/ipykernel_382455/405840175.py:5


2024-07-21 06:01:00,741 [INFO]: Max lines per file are [('/home/cc/dfprofiler/tests/output/simple_test_1MB_128K.pfw.gz', 572)] in /tmp/ipykernel_382455/405840175.py:7
2024-07-21 06:01:00,744 [DEBUG]: Created a batch for /home/cc/dfprofiler/tests/output/simple_test_1MB_128K.pfw.gz from [0, 571] lines in /tmp/ipykernel_382455/2639451476.py:31
2024-07-21 06:01:00,746 [INFO]: Loading 1 batches out of 1 files and has 572 lines overall in /tmp/ipykernel_382455/405840175.py:15


In [31]:
columns = {'pid': "uint64[pyarrow]", 'tid': "uint64[pyarrow]",
           'ts_us': "uint64[pyarrow]", 'dur_sec': "float32[pyarrow]", 
           'freq': "uint64[pyarrow]", 'size_bytes': "uint64[pyarrow]", 'func_id': "string[pyarrow]", 
           'cat': "string[pyarrow]"}

In [32]:
events = pfw_bag.to_dataframe(meta=columns)

In [33]:
events = events.repartition(npartitions=1).persist()
_ = wait(events)

In [34]:
events.head()

,pid,tid,ts_us,dur_sec,freq,size_bytes,func_id,cat
0,381498,381498,0,0.000875,427,<NA>,lseek,c
1,381498,381498,0,0.000007,8,<NA>,ext4_file_open,ext4
2,381498,381498,0,0.000122,32,<NA>,write,sys
3,381498,381498,0,0.000007,2,<NA>,statfs,sys
4,381498,381498,0,0.000015,26,<NA>,fcntl,sys


## Analysis

In [16]:
events.query("func_id.str.contains('write')").compute()

,pid,tid,ts,dur,freq,size,func_id,cat
2,381498,381498,0,0.000122,32,<NA>,write,sys
12,381498,381498,0,0.002792,425,445644800,write,c
34,381498,381498,0,0.178385,108833,<NA>,ext4_da_write_end,ext4
36,381498,381498,0,0.079489,108833,<NA>,ext4_da_write_begin,ext4
37,381498,381498,0,0.000929,425,<NA>,ext4_file_write_iter,ext4
...,...,...,...,...,...,...,...,...
551,381498,381498,26000000,0.000694,111,116391936,write,c
553,381498,381498,26000000,0.045917,28308,<NA>,ext4_da_write_end,ext4
556,381498,381498,26000000,0.000017,3,<NA>,write,sys
559,381498,381498,26000000,0.000239,111,<NA>,ext4_file_write_iter,ext4


In [37]:
total_io = events["size_bytes"].sum().compute() / (1024**3)
total_io

np.float64(31.8439059862867)

In [38]:
read_write_time = events.query("size_bytes > 0").groupby(["pid","tid", "ts_us"])[["freq","dur_sec"]].sum().groupby(["ts_us"]).agg({"freq":sum,"dur_sec":max}).sum().compute()
read_write_time["dur_sec"], read_write_time["freq"]

(1.7103642569854856, 35624.0)

In [39]:
print("Bandwidth is {} GB/s".format(total_io / read_write_time['dur_sec']))

Bandwidth is 18.618201272757847 GB/s


In [44]:
events.query("ts_us == 26000000").compute()

,pid,tid,ts_us,dur_sec,freq,size_bytes,func_id,cat
541,381498,381498,26000000,0.000007,2,<NA>,vfs_statx,vfs
542,381498,381498,26000000,0.000486,111,<NA>,write,sys
543,381498,381498,26000000,0.000022,3,71,write,c
544,381498,381498,26000000,0.000006,1,<NA>,PMPI_Finalize,mpi
545,381498,381498,26000000,0.010438,114,<NA>,rw_verify_area,vfs
546,381498,381498,26000000,0.000006,3,<NA>,vfs_getattr_nosec,vfs
547,381498,381498,26000000,0.000035,6,<NA>,close,c
548,381498,381498,26000000,0.000437,111,<NA>,mark_buffer_dirty,vfs
549,381498,381498,26000000,0.000229,86,<NA>,calloc,c
550,381498,381498,26000000,0.034905,13257,<NA>,free,c


In [41]:
events.query("func_id.str.contains('read')").compute()

,pid,tid,ts_us,dur_sec,freq,size_bytes,func_id,cat
13,381498,381498,0,0.062718,3411,451054842,read,c
26,381498,381498,0,0.000019,14,<NA>,readlinkat,sys
28,381498,381498,0,0.058369,3417,<NA>,read,sys
62,381498,381498,0,0.000014,14,<NA>,vfs_readlink,vfs
76,381498,381498,1000000,0.063713,636,<NA>,read,sys
...,...,...,...,...,...,...,...,...
524,381498,381498,24000000,0.059911,633,<NA>,read,sys
525,381498,381498,25000000,0.061205,639,670040064,read,c
536,381498,381498,25000000,0.060173,639,<NA>,read,sys
554,381498,381498,26000000,0.010651,111,116391936,read,c


In [43]:
events.query("ts_us == 24000000").compute()

,pid,tid,ts_us,dur_sec,freq,size_bytes,func_id,cat
494,381498,381498,24000000,0.034422,366,<NA>,rw_verify_area,vfs
495,381498,381498,24000000,0.002316,366,383778816,write,c
496,381498,381498,24000000,0.00162,366,<NA>,write,sys
497,381498,381498,24000000,0.000208,366,<NA>,lseek,sys
498,381498,381498,24000000,0.000637,366,<NA>,lseek64,c
499,381498,381498,24000000,0.044002,93774,<NA>,mark_buffer_dirty,os_cache
500,381498,381498,24000000,0.034542,93806,<NA>,ext4_da_reserve_space,ext4
501,381498,381498,24000000,0.001454,367,<NA>,mark_buffer_dirty,vfs
502,381498,381498,24000000,0.068048,93881,<NA>,ext4_da_write_begin,ext4
503,381498,381498,24000000,0.067998,93885,<NA>,mark_buffer_dirty,block


In [18]:
events["func_id"].unique().compute()

2024-07-21 04:51:07,174 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle bfa9377f58e4daf68a73b7fbac18eece initialized by task ('shuffle-transfer-bfa9377f58e4daf68a73b7fbac18eece', 0) executed on worker tcp://127.0.0.1:43277
2024-07-21 04:51:07,626 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle bfa9377f58e4daf68a73b7fbac18eece deactivated due to stimulus 'task-finished-1721537467.622656'
2024-07-21 04:51:16,252 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 3e8cd74bb076dcb5eba0feb5ca320dcc initialized by task ('shuffle-transfer-3e8cd74bb076dcb5eba0feb5ca320dcc', 0) executed on worker tcp://127.0.0.1:43277
2024-07-21 04:51:16,666 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 3e8cd74bb076dcb5eba0feb5ca320dcc deactivated due to stimulus 'task-finished-1721537476.663104'
2024-07-21 04:51:48,866 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 53946cd099a01cd1ed69797a273898ab initialized by task ('shuffle-transfer-53946cd099a01cd1

0           mark_page_accessed
1          ext4_da_write_begin
2                       statfs
3                         open
4                        lseek
5                      lseek64
6                   readlinkat
7               posix_memalign
8                         free
9                       openat
10                       close
11                   PMPI_Init
12                       write
13        ext4_file_write_iter
14              PMPI_Comm_size
15                   ftruncate
16                      munmap
17                      fileno
18    vfs_statfs.part.0.isra.0
19                      calloc
20                     realloc
21                  vfs_unlink
22                      mmap64
23      _Z10gen_randomB5cxx11i
24              ext4_file_open
25                        read
26                        mmap
27                      malloc
28           mark_buffer_dirty
29                      open64
30             fileno_unlocked
31                       _init
32      

In [19]:
events.query("cat == 'app'")["func_id"].unique().compute()

0    _Z10gen_randomB5cxx11i
1                     _init
2                     _fini
3                      main
Name: func_id, dtype: string

In [30]:
events.query("cat == 'mpi'")["func_id"].unique().compute()

0         PMPI_Init
1    PMPI_Comm_size
2    PMPI_Comm_rank
3       PMPI_Reduce
4     PMPI_Finalize
Name: func_id, dtype: string

In [45]:
functions = events.groupby(["func_id", "cat", "pid","tid", "ts_us"])[["freq","dur_sec"]].sum().groupby(["func_id", "cat", "ts_us"]).agg({"freq":sum,"dur_sec":max}).groupby([ "cat","func_id"]).sum()
functions = functions.reset_index()
functions.compute()

,cat,func_id,freq,dur_sec
0,app,_Z10gen_randomB5cxx11i,1,0.019031
1,app,_fini,1,0.000004
2,app,_init,1,0.000007
3,app,main,1,0.000005
4,block,mark_buffer_dirty,4135320,3.003395
5,c,calloc,3448,0.011363
6,c,close,1541,0.004051
7,c,fcntl,4,0.00001
8,c,fdopen,2,0.00001
9,c,fileno,7,0.000022


In [46]:
num_writes = functions.query("func_id == 'write' and cat == 'c'")
num_writes.compute()

,cat,func_id,freq,dur_sec
25,c,write,16321,0.105517


In [47]:
num_writes_ext4 = functions.query("func_id.str.contains('ext4_file_write_iter') and cat == 'ext4'")
num_writes_ext4.compute()

,cat,func_id,freq,dur_sec
30,ext4,ext4_file_write_iter,16265,0.035417


In [48]:
num_writes_os_cache = functions.query("func_id.str.contains('mark_buffer_dirty') and cat == 'os_cache'")
num_writes_os_cache.compute()

,cat,func_id,freq,dur_sec
36,os_cache,mark_buffer_dirty,4163872,1.956618


In [49]:
num_reads = functions.query("func_id == 'read' and cat == 'c'")
num_reads.compute()

,cat,func_id,freq,dur_sec
21,c,read,19303,1.604848


In [50]:
num_reads_ext4 = functions.query("func_id.str.contains('read') and cat == 'ext4'")
num_reads_ext4.compute()

,cat,func_id,freq,dur_sec


In [51]:
num_writes_os_cache = functions.query("func_id.str.contains('mark_page_accessed') and cat == 'os_cache'")
num_writes_os_cache.compute()

,cat,func_id,freq,dur_sec
37,os_cache,mark_page_accessed,1,0.000001


In [52]:
min_ts, max_ts = dask.compute(events["ts_us"].min(), events["ts_us"].max())

In [62]:
print(f"Application Time {(max_ts - min_ts) / 1e6}")

Application Time 26.0


In [58]:
total_calls = events.freq.sum().compute()
total_calls

np.int64(21068152)

In [59]:
system_calls = events.query("cat == 'sys'")["freq"].sum().compute()
system_calls

np.int64(55417)

In [61]:
print(f"Events captured per system call {total_calls/system_calls}")

Events captured per system call 380.17489218109966
